# 1. Use a Command Job to create a Machine Learning Model

A Command Job in Azure Machine Learning (Azure ML) is a job type that allows you to execute a Python script, shell command, or any executable in a containerized environment. It is often used to run machine learning training scripts, data preprocessing steps, or custom tasks in Azure ML.

For command job the below arguments are required
- code: Path to the directory containing the script or code files.
- command: The command to execute, often invoking the script (e.g., python train.py).
- inputs: Dictionary of inputs passed to the script (e.g., data paths, hyperparameters).
- outputs: Dictionary of outputs produced by the script (e.g., model files).
- environment: The runtime environment, including dependencies (e.g., Python packages, Docker image).
- compute: The compute target to use (e.g., a virtual machine or compute cluster in Azure ML).
- description: A brief description of the job's purpose.


In [2]:
## Create a handle to Workspace
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
subscription='73f10b45-ec04-427e-a5d6-42ed4e79c537'
resourcegroup='MLRG01'
workspace='AzureMLWS01'

ml_client=MLClient(credential=DefaultAzureCredential(),
subscription_id=subscription,
resource_group_name=resourcegroup,
workspace_name=workspace)

In [3]:
ml_client.workspace_name

'AzureMLWS01'

In [4]:
ws=ml_client.workspaces.get(workspace)
ws.name, ws.resource_group

('AzureMLWS01', 'MLRG01')

# 2. Create the job Env

In [8]:
import os
dependecies_dir='../dependencies'
os.makedirs(dependecies_dir,exist_ok=True)
#Users/mr.snath88/dependencies
## then upload the conda.yaml file

In [7]:
from azure.ai.ml.entities import Environment

custom_job_env = Environment(
    name="sklearn-env",
    description="Environment for sklearn-based training",
    conda_file="../dependencies/conda.yaml",
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
)

custom_job_env=ml_client.environments.create_or_update(custom_job_env)

print(f'Env: {custom_job_env.name} created and the version is {custom_job_env.version}')

Env: sklearn-env created and the version is 7


# 3. Create Training Script

In [14]:
## create directory
script_path='../src'
os.makedirs(script_path,exist_ok=True)

# 4. Configure the command

In [9]:
from azure.ai.ml import command
# Define the command job
# job = command(
#     code="./",  # Path to the directory containing the script
#     command="python training_script_with_argparse.py --data Social_Network_Ads --test_train_ratio 0.2 --estimator random_forest --registered_model_name my_registered_model",
#     environment="azureml:my-custom-environment:1",  # Predefined environment or use your own
#     compute="<your-compute-cluster>",  # Cluster for execution
#     experiment_name="train_with_argparse",
# )

###defining the arguments by dictionary

input_params = {
    "data": "Social_Network_Ads",
    "test_train_ratio": 0.2,
    "estimator": "random_forest",
    "registered_model_name": "Social_mdedia_model",
}

arguments = " ".join([f"--{key} {value}" for key, value in input_params.items()])

job = command(
    code="../src/",  # Path to the directory containing the script
    command=f"python main.py {arguments}",
    environment="AzureML-sklearn-1.0-ubuntu20.04-py38-cpu@latest",  # Predefined environment or use your own
    experiment_name="Social_Media_exp",
    display_name='Social_mdeia_Ads_classification',
)

ml_client.jobs.create_or_update(job)

ResourceNotFoundError: (UserError) System.Net.Http.HttpConnectionResponseContent
Code: UserError
Message: System.Net.Http.HttpConnectionResponseContent

# 5. Creating the endpoints